## Record Linkage Unsupervised Companies

In [1]:
!pip install recordlinkage --quiet
!pip install sklearn --quiet

In [2]:
import numpy as np 
import pandas as pd
import sklearn
import recordlinkage
import warnings
from recordlinkage.index import Full
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [3]:
companies = pd.read_csv("alignedSchemas/companiesAligned.csv")
companies = companies.drop('Unnamed: 0', 1)
companies.head(10)

,name,headquarter,employees,industry,website,ticker,ceo,revenue_M,marketcap_M,shareprice,id
0,GROUPON,"600 W CHICAGO AVE SUITE 400 CHICAGO, IL 60616","1,001 TO 5,000",INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN,NaN,1
1,E-TECHNOLOGIES,AUCKLAND,2 TO 10,INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN,NaN,2
2,YAKSHNA SOLUTIONS,"HERNDON, VA",11 TO 50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,THRU TUBING SOLUTIONS,OKLAHOMA CITY,51 TO 200,"ENERGY, MINING & UTILITIES",NaN,NaN,NaN,NaN,NaN,NaN,4
4,CLEVELAND CLIFFS INC,"CLEVELAND, OH","5,001 TO 10,000",PERSONAL CONSUMER SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,5
5,SISU,LOS ANGELES,11 TO 50,RESTAURANTS & CAFES,NaN,NaN,NaN,NaN,NaN,NaN,6
6,TSMC,"SAN JOSE, CA",51 TO 200,MANUFACTURING,NaN,NaN,NaN,NaN,NaN,NaN,7
7,MATHIS BROTHERS FURNITURE,"OKLAHOMA CITY, OKLAHOMA","1,001 TO 5,000",RETAIL & WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,8
8,IVYTECH SOLUTIONS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,VERTIV,"1050 DEARBORN DRIVE, COLUMBUS, OH 43085, UNITE...","10,000+",TELECOMMUNICATIONS,NaN,NaN,NaN,NaN,NaN,NaN,10


In [4]:
indexer = recordlinkage.Index()
indexer.block('name')
pairs = indexer.index(companies, companies)

In [5]:
print(len(pairs))

178227


In [6]:
train_pairs = pairs[:85000]
test_pairs = pairs[85000:]

## Training Unsupervised

In [7]:
compare = recordlinkage.Compare()

compare.string('name', 'name', label="name", threshold=0.80)
compare.exact('employees', 'employees', label='employees')
compare.string('website', 'website', method='jarowinkler', label="website", threshold=0.85)
compare.string('ticker', 'ticker', method='jarowinkler', label="ticker", threshold=0.95)
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo", threshold=0.90)

training_features = compare.compute(train_pairs, companies, companies)
training_features['score'] = training_features.loc[:, 'name':'ceo'].sum(axis=1)

In [8]:
training_matches = training_features[training_features.sum(axis=1) > 2].reset_index()
training_matches['score'] = training_matches.loc[:, 'name':'ceo'].sum(axis=1)
toDrop = ['name', 'employees', 'website', 'ticker', 'ceo', 'score']
#toDropTrainingFeatures = ['score']
training_matches = training_matches.drop(toDrop, axis=1)
#training_features = training_features.drop(toDropTrainingFeatures, axis=1)
training_matches.head()

,level_0,level_1
0,0,0
1,44061,44061
2,1,1
3,2,2
4,3,3


In [9]:
training_matches = pd.MultiIndex.from_frame(training_matches) #matches conversion

In [10]:
training_features.head()

name  employees  website  ticker  ceo  score
0     0       1.0          1      0.0     0.0  0.0    2.0
      20812   1.0          0      0.0     0.0  0.0    1.0
      44061   1.0          0      0.0     0.0  0.0    1.0
      62588   1.0          0      0.0     0.0  0.0    1.0
20812 0       1.0          0      0.0     0.0  0.0    1.0

In [11]:
len(training_features)

85000

In [12]:
len(training_matches)

48324

#### Classifier

In [13]:
classifier = recordlinkage.KMeansClassifier()
classifier.fit(training_features)#, training_matches)

## Testing And Evaluation

In [14]:
compare = recordlinkage.Compare()

compare.string('name', 'name', label="name", threshold=0.80)
compare.exact('employees', 'employees', label='employees')
compare.string('website', 'website', method='jarowinkler', label="website", threshold=0.85)
compare.string('ticker', 'ticker', method='jarowinkler', label="ticker", threshold=0.95)
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo", threshold=0.90)

test_features = compare.compute(test_pairs, companies, companies)
test_features['score'] = test_features.loc[:, 'name':'ceo'].sum(axis=1)

In [15]:
test_matches = test_features[test_features.sum(axis=1) > 2].reset_index()
test_matches['score'] = test_matches.loc[:, 'name':'ceo'].sum(axis=1)
toDrop = ['name', 'employees', 'website', 'ticker', 'ceo', 'score']
#toDropTestFeatures = ['score']
test_matches = test_matches.drop(toDrop, axis=1)
#test_features = test_features.drop(toDropTestFeatures, axis=1)
test_matches.head(10)

,level_0,level_1
0,43241,10172
1,43241,25041
2,43241,43241
3,40299,40299
4,41191,41191
5,41018,41018
6,44268,44268
7,39917,39917
8,10178,10178
9,25047,25047


In [16]:
test_matches = pd.MultiIndex.from_frame(test_matches) #matches conversion

In [17]:
predictions = classifier.predict(test_features)

In [18]:
# return the confusion matrix
confusion_matrix = recordlinkage.confusion_matrix(test_matches, predictions, len(test_features))
print('confusion matrix')
print(confusion_matrix)

# compute the F-score for this classification
fscore = recordlinkage.fscore(confusion_matrix)
print('\n\nfscore', fscore)
recall = recordlinkage.recall(test_matches, predictions)
print('recall', recall)
precision = recordlinkage.precision(test_matches, predictions)
print('precision', precision)
accuracy = recordlinkage.accuracy(test_matches, predictions, len(test_features))
print('accuracy', accuracy)

confusion matrix
[[33714     0]
 [    0 59513]]


fscore 1.0
recall 1.0
precision 1.0
accuracy 1.0
